# With ChatModel

In [58]:
import json

from datetime import datetime

from langchain.chat_models import ChatOpenAI
from langchain.prompts import load_prompt
from langchain.schema import StrOutputParser
from langchain_core.output_parsers import JsonOutputParser

In [59]:
def save_result(output, prompt_type, model, init, best_practice, format_prompt, output_parser):
    now = datetime.now()
    date = now.strftime("%Y-%m-%d")
    time = now.strftime("%H:%M:%S")

    result = {}
    result['model'] = model
    result['creation_timestamp'] = date + ' ' + time
    result['prompts'] = {
        'init_uri': init,
        'best_practice_uri': best_practice,
        'formatter_uri': format_prompt}
    result['output_parser'] = output_parser 
    result['output'] = output

    with open('results/'+prompt_type+'/output_'+date+'_'+time+'.json', 'w') as f:
        json.dump(result, f, indent=4)

In [60]:
def StringToJSON(string):
    result = string

    try:
        if ((string[0] != "{") and (string[0] != "[")):
            string = "{"+string+"}"

        result = json.loads(string)

    except Exception as e:
        print("JSON formatting went wrong. The result returns as a string.")
        
    finally:
        return result

In [61]:
model_name = 'gpt-4'

model = ChatOpenAI(
    model=model_name,
    api_key=open('api.txt', 'r').read(),
    temperature=0
)

In [62]:
sample_name = "suggestion_sample.json"
with open('./data/json/last_row/'+sample_name) as f:
    json_sample = json.load(f)

## Summary Length

In [65]:
init_prompt_name = 'initPrompt_V1.2.0.json'
init_prompt = load_prompt('prompts/init/'+init_prompt_name)

bestPractice_prompt_name = 'summaryLengthPrompt_V2.3.0.json'
bestPractice_prompt = load_prompt('prompts/summary/'+bestPractice_prompt_name).format(min=39, max=70)

format_prompt_name = 'summaryLengthFormatPrompt_V1.0.0.json'
format_prompt = load_prompt('prompts/summary/'+format_prompt_name)

In [66]:
chain1 = init_prompt | model | StrOutputParser()
chain2 = (
    {"revised_ticket": chain1}
    | format_prompt
    | model
    | JsonOutputParser()
)

In [71]:
output = await chain2.ainvoke({"role":"Software Engineer", "best_practice":bestPractice_prompt, "ticket":json_sample})
output

{'summary_old': 'Project Summaries',
 'summary_new': 'Single page display of project components and their issue status'}

In [69]:
# should be between 39 and 70
len('Single page display of project components and their issue status')

64

In [72]:
save_result(output, 'summary', model_name, init_prompt_name, bestPractice_prompt_name, format_prompt_name, 'JsonOutputParser')

In [31]:
# # maybe usefull for later
# def run(model_name, init_prompt_name, sumLen_prompt_name):
#     model = ChatOpenAI(
#         model=model_name,
#         api_key=open('api.txt', 'r').read(),
#         temperature=0
#     )
#     with open('./data/json/last_row.json') as f:
#         json_sample = json.load(f)

#     init_prompt = load_prompt('prompts/init/'+init_prompt_name)
#     sumLen_prompt = load_prompt('prompts/summary/'+sumLen_prompt_name).format(min= 6, max= 10)
    
#     chain = init_prompt | model | StrOutputParser()

#     output = chain.invoke({"best_practice": sumLen_prompt,"ticket": json_sample})

#     save_result(output, 'summary', model_name, init_prompt_name, sumLen_prompt_name)  

# run('gpt-4', 'initPrompt_V1.0.0.json', 'summaryLengthPrompt_V1.0.1.json')  

## Description Length (maybe not usefull)

In [15]:
init_prompt_name = 'initPrompt_V1.1.2.json'
init_prompt = load_prompt('prompts/init/'+init_prompt_name)

bestPractice_prompt_name = 'descriptionLengthPrompt_V2.7.0.json'
bestPractice_prompt = load_prompt('prompts/description/'+bestPractice_prompt_name).format(min= 210, max= 850)

chain = init_prompt | model | StrOutputParser()

In [16]:
output = chain.invoke({"role":"Software Engineer", "best_practice":bestPractice_prompt, "ticket":json_sample})
output

'The description of the ticket is: "A single page displaying all the different components for a project, and the status of different issues for the component. Could work with either open bugs per component, or planned features per component. (For all versions or for a specific version) See http://www7b.software.ibm.com/webapp/wsdd/wasServlet3?client=form10&SID=1013344492744 for how it might possibly look (I like the graphical status indicators)". \n\nThis description is too short as per the best practice of having between 210 and 850 characters. \n\nRecommended description: "The issue pertains to the development of a single page that displays all the different components of a project, along with the status of various issues related to each component. The page should be designed to work with either open bugs per component or planned features per component. This functionality should be applicable for all versions of the project or for a specific version, as required. For a visual represe

In [17]:
len(output)

1258

In [18]:
save_result(output, 'description', model_name, init_prompt_name, bestPractice_prompt_name, 'StrOutputParser')

## Field Update

In [19]:
init_prompt_name = 'initPrompt_V1.1.2.json'
init_prompt = load_prompt('prompts/init/'+init_prompt_name)

bestPractice_prompt_name = 'updatePrompt_V1.12.2.json'
bestPractice_prompt = load_prompt('prompts/update/'+bestPractice_prompt_name).format()

In [20]:
chain = init_prompt | model | StrOutputParser()

In [21]:
output = chain.invoke({"role":"Software Engineer", "best_practice":bestPractice_prompt, "ticket":json_sample})

In [22]:
output_json = StringToJSON(output)

In [23]:
save_result(output_json, 'update', model_name, init_prompt_name, bestPractice_prompt_name, 'StrOutputParser')

## Toxic Speech Detection

In [80]:
with open('./data/json/last_row/last_row_toxic.json') as f:
    toxic_sample = json.load(f)

In [92]:
init_prompt_name = 'initPrompt_V1.1.2.json'
init_prompt = load_prompt('prompts/init/'+init_prompt_name)

bestPractice_prompt_name = 'toxicSpeechPrompt_V1.5.0.json'
bestPractice_prompt = load_prompt('prompts/toxicSpeech/'+bestPractice_prompt_name).format()

format_prompt_name = 'toxicSpeechFormatPrompt_V1.1.0.json'
format_prompt = load_prompt('prompts/toxicSpeech/'+format_prompt_name)

In [93]:
chain1 = init_prompt | model | StrOutputParser()
chain2 = (
    {"revised_ticket": chain1}
    | format_prompt
    | model
    | JsonOutputParser()
)

In [94]:
output = await chain2.ainvoke({"role":"Content Moderator", "best_practice":bestPractice_prompt, "ticket":toxic_sample})
output

{'comment': 'Issue was fixed in latest Sourcetree versions you idiot. Use your brain and use Sourcetree 3.4.5 >',
 'author': 'Oleksandr Naumenko',
 'datetime': '2021-11-18 15:12:59.861000+00:00',
 'recommendation': 'The issue has been resolved in the latest Sourcetree versions. I recommend using Sourcetree 3.4.5 or later.'}

In [95]:
save_result(output, 'toxicSpeech', model_name, init_prompt_name, bestPractice_prompt_name, format_prompt_name, 'JsonOutputParser')

## Bug Report Structure

In [23]:
bugReportStrucutre_prompt_name = 'bugReportStructurePrompt_V2.1.0.json'
bugReportStrucutre_prompt = load_prompt('prompts/bugReportStructure/'+bugReportStrucutre_prompt_name)

In [24]:
chain = bugReportStrucutre_prompt | model | StrOutputParser()

In [25]:
output = chain.invoke({"bug_report":json_sample})

In [26]:
save_result(output, 'bugReportStructure', model_name, '-', bugReportStrucutre_prompt_name, 'StrOutputParser')

## Internationalization

In [50]:
with open('./data/json/last_row/last_row_german.json') as f:
    german_sample = json.load(f)

In [51]:
init_prompt_name = 'initPrompt_V1.2.0.json'
init_prompt = load_prompt('prompts/init/'+init_prompt_name)

bestPractice_prompt_name = 'internationalizationPrompt_V2.1.0.json'
bestPractice_prompt = load_prompt('prompts/internationalization/'+bestPractice_prompt_name).format()

format_prompt_name = 'internationalizationFormatPrompt_V1.2.0.json'
format_prompt = load_prompt('prompts/internationalization/'+format_prompt_name)

In [53]:
chain1 = init_prompt | model | StrOutputParser()

chain2 = (
    {"revised_ticket":chain1}
    | format_prompt
    | model
    | JsonOutputParser()
    )

In [56]:
output = await chain2.ainvoke({"role":"Software Engineer", "best_practice":bestPractice_prompt, "ticket":german_sample})
output

{'Summary': 'Sourcetree crashes',
 'Description': 'The Sourcetree crashes every time I open it, I have uninstalled and installed several times',
 'VersionsAffected': '3.4.4',
 'IssueType': 'Bug',
 'Project': 'Sourcetree for Windows',
 'Components': 'Git',
 'CreatedDate': '2021-09-13T05:48:08.000+0000',
 'ResolvedDate': '2021-11-18T15:12:59.000+0000',
 'Status': 'Closed',
 'Priority': 'Low',
 'Creator': 'Shefali Bhandary',
 'Reporter': 'Shefali Bhandary',
 'Resolution': 'Cannot Reproduce',
 'IssueLinks': None,
 'Labels': None,
 'VersionsFixed': None,
 'Assignee': None,
 'TimeSpent': None,
 'Comments': [{'Author': 'Vipin Yadav',
   'Created': '2021-09-13 06:12:40.315000+00:00',
   'Comment': 'Please try to install latest release of sourcetree 3.4.6 and let us know if you are still facing same issue.'},
  {'Author': 'Shefali Bhandary',
   'Created': '2021-09-13 06:56:53.753000+00:00',
   'Comment': 'In our software portal on 3.2.6 is available'},
  {'Author': 'Vipin Yadav',
   'Created': 

In [57]:
save_result(output, 'internationalization', model_name, init_prompt_name, bestPractice_prompt_name, format_prompt_name, 'JsonOutputParser')